# 1. 라이브러리

In [1]:
import pandas as pd
import numpy as np
import os
import re
import psycopg2
import psycopg2.extras

from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import font_manager
from wordcloud import WordCloud

# 2. DB 연결

In [6]:
conn = psycopg2.connect(
    host="xx.xx.xx.xx",
    port="5432",
    dbname="postgres",
    user="postgres",
    password="password"
)
cur = conn.cursor()

# 3. 데이터 정제

## 3.1. 데이터 가져오기

In [11]:
batch_size = 10000
last_id = 0
dfs = []

while True:
    query = """
        SELECT id, streamer_id, user_id, msg, ts
        FROM chat_logs
        WHERE id > %s
        ORDER BY id ASC
        LIMIT %s;
    """
    cur.execute(query, (last_id, batch_size))
    rows = cur.fetchall()
    
    if not rows:
        break
    
    df = pd.DataFrame(rows, columns=["id", "streamer_id", "user_id", "msg", "ts"])
    dfs.append(df)
    
    last_id = rows[-1][0]   # 마지막 row의 id를 다음 루프 시작점으로
    print(f"{last_id} 까지 읽음 ({len(rows)} rows)")

chat_df = pd.concat(dfs, ignore_index=True)

print(final_df.head())
print("총 row 수:", len(final_df))

10201 까지 읽음 (10000 rows)
20202 까지 읽음 (10000 rows)
30202 까지 읽음 (10000 rows)
40206 까지 읽음 (10000 rows)
50206 까지 읽음 (10000 rows)
60207 까지 읽음 (10000 rows)
70207 까지 읽음 (10000 rows)
80207 까지 읽음 (10000 rows)
90207 까지 읽음 (10000 rows)
100207 까지 읽음 (10000 rows)
110207 까지 읽음 (10000 rows)
120207 까지 읽음 (10000 rows)
130207 까지 읽음 (10000 rows)
140207 까지 읽음 (10000 rows)
150207 까지 읽음 (10000 rows)
160207 까지 읽음 (10000 rows)
170207 까지 읽음 (10000 rows)
180207 까지 읽음 (10000 rows)
190207 까지 읽음 (10000 rows)
200208 까지 읽음 (10000 rows)
210209 까지 읽음 (10000 rows)
220279 까지 읽음 (10000 rows)
230279 까지 읽음 (10000 rows)
240279 까지 읽음 (10000 rows)
250282 까지 읽음 (10000 rows)
260282 까지 읽음 (10000 rows)
270282 까지 읽음 (10000 rows)
280282 까지 읽음 (10000 rows)
290282 까지 읽음 (10000 rows)
300282 까지 읽음 (10000 rows)
310282 까지 읽음 (10000 rows)
320282 까지 읽음 (10000 rows)
330282 까지 읽음 (10000 rows)
340282 까지 읽음 (10000 rows)
350282 까지 읽음 (10000 rows)
360282 까지 읽음 (10000 rows)
370282 까지 읽음 (10000 rows)
380282 까지 읽음 (10000 rows)
390282 까지 읽음 (10000 r

## 3.2. 데이터 프레임

In [12]:
chat_df = final_df

In [15]:
chat_df.columns = ["id", "streamer_id", "nickname", "chat", "time"]

In [16]:
chat_df["chat"].values

array(['1132ㄷㄷ', 'ㄷㄷㄷㄷㄷ', '캬', ..., '사이퍼 백퍼 돈다', '부상이슈', '안약 느시죠 안약'],
      dtype=object)

In [17]:
chat_df.dropna(ignore_index=True, inplace=True)

## 3.3. 텍스트 정제

In [113]:
def simple_tokenizer(text):
    # 한국어 단어만 처리
    text = re.sub(r"[^가-힣\s]", " ", text)
    
    # 공백 기준으로 분리
    tokens = text.split()
    return tokens

# 테스트
print(simple_tokenizer("ㅋㅋㅋㅋ 이건 ㄹㅇ 대박 ㅠㅠ!! 헐ㅋㅋ{:dsd:}{:dsd:}{:ddsd:}"))

['이건', '대박', '헐']


In [ ]:
# 불용어 처리
with open('stopwords-ko.txt', 'r', encoding='utf-8') as f:
    stopwords = f.read().splitlines()

# 4. 시각화

## 4.1. 스트리머별 채팅 비율

In [114]:
id_to_name = {
    "75cbf189b3bb8f9f687d2aca0d0a382b": "한동숙",
    "3497a9a7221cc3ee5d3f95991d9f95e9": "랄로",
    "0b33823ac81de48d5b78a38cdbc0ab94": "울프",
    "c7ded8ea6b0605d3c78e18650d2df83b": "괴물쥐",
    "bb382c2c0cc9fa7c86ab3b037fb5799c": "침착맨",
    "a7e175625fdea5a7d98428302b7aa57f": "탬탬버린",
    "45e71a76e949e16a34764deb962f9d9f": "아야츠노 유니",
    "42597020c1a79fb151bd9b9beaa9779b": "파카",
    "b5ed5db484d04faf4d150aedd362f34b": "강지",
    "7ce8032370ac5121dcabce7bad375ced": "풍월량",
    "bdc57cc4217173f0e89f63fba2f1c6e5": "다주",
    "4325b1d5bbc321fad3042306646e2e50": "아카네 리제",
    "a6c4ddb09cdb160478996007bff35296": "아라하시 타비",
    "b044e3a3b9259246bc92e863e7d3f3b8": "시라유키 히나",
    "458f6ec20b034f49e0fc6d03921646d2": "서새봄냥 SEBOM",
    "64d76089fba26b180d9c9e48a32600d9": "텐코 시부키",
    "4515b179f86b67b4981e16190817c580": "네네코 마시로",
    "1c231568d0b13de5703b3f6a5e86dc47": "삼식123",
    "516937b5f85cbf2249ce31b0ad046b0f": "아오쿠모 린",
    "4d812b586ff63f8a2946e64fa860bbf5": "하나코 나나",
}

vc = chat_df["streamer_id"].value_counts(normalize=True) * 100

vc.index = vc.index.map(lambda x: id_to_name.get(x, x))

print(vc)

streamer_id
한동숙           13.808974
아야츠노 유니       13.058395
울프            12.372059
텐코 시부키        11.603367
괴물쥐            9.864865
아카네 리제         7.925424
강지             6.959540
서새봄냥 SEBOM     4.731359
네네코 마시로        4.437713
시라유키 히나        3.219673
하나코 나나         2.857589
아오쿠모 린         2.529501
다주             2.292513
침착맨            1.509902
풍월량            1.482688
탬탬버린           1.346081
삼식123          0.000357
Name: proportion, dtype: float64


## 4.2. 스트리머별 상위 N개 단어

In [101]:
chat_df["token"] = chat_df["chat"].apply(simple_tokenizer)

In [102]:
result = chat_df.groupby('streamer_id')['token'].agg(lambda x: [item for sublist in x for item in sublist]).reset_index()

In [105]:
documents = result['token'].apply(lambda x : ' '.join(x)).values
streamers = result["streamer_id"]

In [106]:
vectorizer = TfidfVectorizer(stop_words=stopwords)
X = vectorizer.fit_transform(documents)
df_tfidf = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out(), index=streamers)

top_30_words = {}
for streamer in df_tfidf.index:
    top_30 = df_tfidf.loc[streamer].sort_values(ascending=False).head(30)
    top_30_words[streamer] = top_30.index.tolist()

for streamer, words in top_30_words.items():
    print(f"{id_to_name[streamer]}의 상위 30개 단어: {', '.join(words)}")

/home/kse/anaconda3/envs/chzzk/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['sebom', '가서', '같은', '것과', '결과에', '결론을', '관계가', '관련이', '그런', '그럼에도', '그렇게', '그에', '그치지', '김에', '까닭에', '나나', '낫다', '네네코', '년도', '논하지', '누가', '다시', '달려', '대로', '대해', '되는', '되다', '되어', '들면', '들자면', '듯하다', '따르는', '따름이다', '따지지', '때가', '리제', '마시로', '만은', '만이', '많은', '말하면', '말할것도', '몰라도', '몰랏다', '못하다', '미치다', '바꾸어서', '바꿔', '밖에', '방면으로', '보면', '보아', '부류의', '비길수', '비추어', '뿐만', '사람들', '상대적으로', '생각이다', '서새봄냥', '서술한바와같이', '시라유키', '시부키', '쓰여', '아니다', '아니라', '아라하시', '아야츠노', '아오쿠모', '아카네', '안다', '안된다', '않고', '않기', '않는다면', '않다', '않다면', '않도록', '않으면', '알겠는가', '어쩔수', '없고', '없다', '예를', '외에', '요만한', '우에', '위에서', '유니', '이렇게', '이로', '이르다', '이와', '이유는', '인하여', '임에', '점에서', '정도에', '정도의', '종합한것과같이', '주저하지', '줄은', '지경이다', '타비', '텐코', '틀림없다', '편이', '하고', '하기', '하기만', '하나코', '하는', '하는것만', '하는것이', '하다'

울프의 상위 30개 단어: 라이즈, 바드, 서넌, 너무, 농심, 젠지, 유나라, 이게, 사건, 이건, 발생, 이거, 시비르, 라이즈의, 한화, 선언, 레전드, 미드, 빅토르, 구나라, 울프님, 아타칸, 코르키, 실험실, 비디디, 사이온, 어어, 어우, 울프야, 티원
삼식123의 상위 30개 단어: 힣힣힣힣힣, 가가, 가가가, 가가가가, 가가고일, 가가깝나, 가가렌, 가가마게, 가가매, 가가매여, 가가매요, 가가메요, 가가미, 가가용, 가가정령의형상이, 가가주고, 가가지고, 가간다, 가갈갱, 가감없이, 가감이, 가감한, 가감해, 가감해요, 가감해용, 가감했다, 힙하네, 힙하다, 힙하면, 힙하잖아
아카네 리제의 상위 30개 단어: 마망, 그러게, 리제, 광증, 광증이, 아카네, 프리스크, 으지, 트리비, 언다인, 어어, 메타톤, 아스리엘, 리제는, 어우, 나데나데, 두둥탁, 리제야, 티탄, 뭐야, 오오, 아스고어, 아오, 리제가, 그건, 오렌지, 그러네, 샌즈, 갱플, 악하네
네네코 마시로의 상위 30개 단어: 안냐냐, 시로, 오오, 나이스, 어어, 어우, 좋다, 대포, 그러게, 너무, 오우, 달려, 매직, 까비, 화이팅, 찌로, 굿굿, 뭐야, 가보자, 제발, 만통, 길이, 오옹, 가자, 풍선, 피크, 황버, 추스, 이건, 이게
서새봄냥 SEBOM의 상위 30개 단어: 복돌이, 아이언, 너무, 새봄님, 리븐, 새봄추, 봄추, 서새봄, 봄하, 이제, 커마, 저거, 이거, 이게, 트타, 애니비아, 많이, 베이가, 그건, 유미, 에이미, 복도링, 지금, 헤드셋, 뭐야, 아님, 어우, 리븐이, 말파, 블츠
아야츠노 유니의 상위 30개 단어: 유니, 덕분이, 유니야, 아내임, 유니가, 스쿼트, 안녕하시지, 유니는, 초장, 오오, 우승, 그건, 너무, 감사합니다, 대유니, 이건, 흐흐흐, 이게, 역시, 유니우승, 아르냥, 꼴든, 스튜, 좋은데, 유니유니, 드가자, 잘하네, 오늘, 잘자시지, 흰색
하나코 나나의 상위 30개 단어: 나나, 나나야, 오오, 나나는, 어어, 정나나

## 4.3. 상위 단어 워드클라우드

In [108]:
FONT_PATH = "/usr/share/fonts/truetype/nanum/NanumGothic.ttf"
nanum_font = font_manager.FontProperties(fname=FONT_PATH)

mpl.rcParams['font.family'] = nanum_font.get_name()
mpl.rcParams['axes.unicode_minus'] = False

print("현재 적용된 폰트:", nanum_font.get_name())

현재 적용된 폰트: NanumGothic


In [112]:
out_dir = "./wordclouds"
os.makedirs(out_dir, exist_ok=True)

def make_wordcloud_for_series(s, title, path, top_n):
    top = s.sort_values(ascending=False).head(top_n)
    top = top[top > 0]
    if top.empty:
        print(f"{title} 단어 없음")
        return
    
    wc = WordCloud(
        width=800,
        height=600,
        background_color="white",
        font_path=FONT_PATH,
    )
    
    img = wc.generate_from_frequencies(top.to_dict())
    plt.figure(figsize=(10, 6))
    plt.imshow(img, interpolation="bilinear")
    plt.axis("off")
    plt.title(title)
    plt.tight_layout()
    plt.savefig(path, dpi=150, bbox_inches="tight")
    plt.close()


results = []
for streamer_id in df_tfidf.index:
    name = id_to_name.get(streamer_id, streamer_id)
    title = f"{name} (TF-IDF WordCloud)"
    out_path = os.path.join(out_dir, f"{name}_wordcloud.png")
    make_wordcloud_for_series(df_tfidf.loc[streamer_id], title, out_path, 100)

findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
/tmp/ipykernel_15876/3050986005.py:32: UserWarning: Glyph 50872 (\N{HANGUL SYLLABLE UL}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_15876/3050986005.py:32: UserWarning: Glyph 54532 (\N{HANGUL SYLLABLE PEU}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfo

삼식123 (TF-IDF WordCloud) 단어 없음


findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
/tmp/ipykernel_15876/3050986005.py:32: UserWarning: Glyph 50500 (\N{HANGUL SYLLABLE A}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_15876/3050986005.py:32: UserWarning: Glyph 52852 (\N{HANGUL SYLLABLE KA}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_15876/3050986005.py:32: UserWarning: Glyph 45348 (\N{HANGUL SYLLABLE NE}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_15876/3050986005.py:32: UserWarning: Glyph 47532 (\N{HANGUL SYLLABLE RI}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/t